In [1]:
try:
    from elasticsearch import Elasticsearch,helpers 
    from pprint import pprint
    import uuid 
    import jellyfish as jf
    import phonetics 
    import time
    import pickle
    import json
    print("All Modules successfully loaded!")
except Exception as e:
    print("Loading modules failed : ", e)


All Modules successfully loaded!


In [2]:
# !pip install 

In [3]:
with open("MP_data", 'rb') as f:
    data = pickle.load(f)

In [4]:
es = Elasticsearch('10.237.26.117', port=9200)

In [5]:
# check if connected to elasticSearch
if es.ping():
    print("connected to ES")
else:
    print("Connection to ES failed")

connected to ES


In [6]:

INDEX = "mp_data"
MAPPING  = "electionData"

In [7]:
# get all index in ES

es_indices = es.indices.get_alias("*")
# for idx in es_indices:
#     print(idx)
    
if INDEX not in es_indices:
    print("Index does not exists ------- Creating now")
    es.indices.create(index=INDEX, request_timeout=30)

else:
    print('Index already exists')

print("index logic done")

Index already exists
index logic done


In [8]:
def findES_Doc(entityName,state, constituency):
#     "from" : 0, "size" : 10,
    return  {
        "size": 5,
        "query":
        {
            "bool":
            {
                "should":
                [
                    {
                        "multi_match" :
                         {
                            "query":    entityName,
                            "fields": [ "Name", "Alias" ]
                         }
                    }

                ],
                "must":
                [
                    { 
                        "terms":  { "State.keyword": state }
                    }
                    
                ]
            }
    
                       
                
        }
    }

In [9]:
# def match(doc, candidate):
#     if doc['Name'] != candidate['_source']['Name'] and doc['Name'] not in candidate['_source']['Alias']:
#         return False
#     if doc['State'][0] != candidate['_source']['State']
#         return False
    
#     for history in candidate['Electoral_Info']:
#         if doc['Constituency'] == history['Constituency']:
#             return True
        
#     return False
    
    

In [10]:
def createDoc(doc):
    mp = dict()
    mp['Name'] = doc['Name']
    mp['Alias'] =  doc['Alias']
    mp['State'] = doc['State'][0]
    mp['Electoral_Info'] = [{'Constituency':doc['Constituency'] , 'Party': doc['Party'] ,
                             'Position': doc['Position'], 'Year':doc['Year']}]
    
    return mp


In [12]:
ls2004_09_14 = []
for doc in data:
    mp = createDoc(doc)
    ls2004_09_14.append(mp)
    
print(len(ls2004_09_14))

19690


In [13]:
resp = helpers.bulk(
es,
ls2004_09_14,
index = INDEX,
doc_type = "_doc"
)

/home/vivek/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [14]:
print ("helpers.bulk() RESPONSE:", resp)
print ("helpers.bulk() RESPONSE:", json.dumps(resp, indent=4))

helpers.bulk() RESPONSE: (19690, [])
helpers.bulk() RESPONSE: [
    19690,
    []
]


In [15]:
# inserting 2019 data as not present in Graph DB

with open('ls2019data', 'rb') as f:
    ls2019 = pickle.load(f)
    
len(ls2019)


8026

In [16]:
resp2019 = helpers.bulk(
es,
ls2019,
index = INDEX,
doc_type = "_doc"
)

    
print ("helpers.bulk() RESPONSE:", resp2019)
print ("helpers.bulk() RESPONSE:", json.dumps(resp2019, indent=4))

helpers.bulk() RESPONSE: (8026, [])
helpers.bulk() RESPONSE: [
    8026,
    []
]


In [ ]:
res = es.search(index=INDEX, doc_type="_doc", body = {
'size' : 10000,
'query': {
    'match_all' : {}
}
})
len(res['hits']['hits'])

In [ ]:
# result = es.search(index=INDEX, body={"query":{"match_all":{}}})
# len(result['hits']['hits'])

In [ ]:
# print(result)

In [ ]:
# delete the index
# print(es.indices.delete(index=INDEX, ignore=[400,404]))